# CODE

First, lets import all the necessary packages

In [1]:
import h5py
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

c:\users\shash\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
c:\users\shash\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


Global variables

In [2]:
# File paths
TRAIN_CSV = 'C:\\Users\\shash\\OneDrive\\Desktop\\project\\train.csv'
#TRAIN_CSV = 'C:\\Users\\shash\\OneDrive\\Desktop\\project\\test.csv'
TEST_CSV = 'C:\\Users\\shash\\OneDrive\\Desktop\\project\\test.csv'
EMBEDDING_FILE = 'C:\\Users\\shash\\OneDrive\\Desktop\\project\\GoogleNews-vectors-negative300.bin.gz'
MODEL_SAVING_DIR = 'C:\\Users\\shash\\OneDrive\\Desktop\\project\\'

Create embedding matrix

In [4]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)
            
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

c:\users\shash\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:76: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Prepare training and validation data

In [5]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max(),
                     test_df.question1.map(lambda x: len(x)).max(),
                     test_df.question2.map(lambda x: len(x)).max())

# Split to train validation
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
#Y = train_df['test_id']
Y = train_df['is_duplicate']

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

Build the model

In [11]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25

#def exponent_neg_manhattan_distance(left, right):
    #'''Helper function for the similarity estimate of the LSTMs outputs'''
    #return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: K.exp(-K.sum(K.abs(x[0]-x[1]), axis=1, keepdims=True)), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

c:\users\shash\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:28: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
c:\users\shash\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:42: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2305796 samples, validate on 40000 samples
Epoch 1/25
1922496/2305796 [========================>.....] - ETA: 38:16 - loss: 1834045132805.0452 - acc: 5.2016e-07

KeyboardInterrupt: 

Plotting the results

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
malstm.save('my_model.h5')

In [12]:
model = load_model('my_model.h5')
model.summary()

c:\users\shash\appdata\local\programs\python\python36\lib\site-packages\keras\engine\topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 213)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 213)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 213, 300)     36396000    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 50)           70200       embedding_1[0][0]                
          

In [38]:
'''import numpy as np
import operator
#model = load_model('my_model.h5')
#print(model)
#print(X_train['left'][0])
#print(X_train['right'][0].shape)
a = np.ones(213)
a = np.int16(a)
#a = np.matrix(a)
#rint(type(X_train['left'][0]))
#ype(a)
#rint(Y_train[0:50])
#print(X_test['left'])
#print(Y_validation[0:500])
#print(X_validation['left'].shape)
#print(Y_train)
print(X_train['left'].shape)
a=(list(model.predict([X_train['left'][0:2305796], X_train['right'][0:2305796]])))
m=max(a)
print(a.index(m))
'''

(2305796, 213)
2689


In [266]:
#np.savetxt('file.out', c.values, delimiter=',')
r=list(ans)
lq=ln.reshape(len(li),1)
print((lq.shape))
print((ans.shape))
data=np.concatenate((lq,ans),axis=1)
print(ans)
np.savetxt('final_data.csv',data,fmt='%.4f',delimiter=',',header="test_id,is_duplicate")

(2345796, 1)
(2345796, 1)
[[0.1084455 ]
 [0.1922526 ]
 [0.19555989]
 ...
 [0.0071587 ]
 [0.04845008]
 [0.00161017]]


In [113]:
#X_train['left'][3]
#X_test['left']
print(type(X_test['left']))
q=X_test['left']
pad_sequences(q, maxlen=213)
print(X_validation['left'].shape)

<class 'pandas.core.series.Series'>
(40000, 213)


In [121]:
q=pad_sequences(q, maxlen=213)

In [147]:
r=pad_sequences(X_test['right'],maxlen=213)

In [160]:
ans=model.predict([q,r])

In [179]:
li=list(range(0,2345796))
print(len(li))
print(ans.shape)

2345796
(2345796, 1)


In [187]:
print(type(li))
from numpy import array
ln=array(li)
type(ans)

<class 'list'>


numpy.ndarray